## Movielens 영화 추천 실습

In [1]:
# MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
# 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

# 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
# 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
# 별점을 시청횟수로 해석해서 생각하겠습니다.
# 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리

In [2]:
# Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [3]:
import pandas as pd
import os
data_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
data_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv(data_file_path, sep='::', names=data_cols, engine='python')
orginal_data_size = len(data)

In [4]:
# 필요없는 항목은 삭제한다.
data.drop('timestamp', axis=1, inplace=True)

In [5]:
data.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [6]:
# 3점 이상만 남깁니다.
data = data[data['rating']>=3]
filtered_data_size = len(data)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [7]:
# data 컬럼의 이름을 count로 바꿉니다.
data.rename(columns={'rating':'count'}, inplace=True)

In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
# 여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 
# MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2) 분석해 봅시다.

In [10]:
# ratings에 있는 유니크한 영화 개수
# pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.
data['movie_id'].nunique() # 영화갯수

3628

In [11]:
# data에 있는 유니크한 사용자 수
data['user_id'].nunique()

6039

In [12]:
# 가장 인기있는 영화 30개(인기순)
# 영화제목과 같이 보여주기 위해서 movies에서 영화제목을 가져옴
# data1 = data.copy()
data = pd.merge(data, movies[['movie_id', 'title']], how='left', on='movie_id')

In [13]:
data.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


In [14]:
# 가장 많이 본 영화 순서대로 조회해 보기
data.groupby(['movie_id', 'title'])['user_id'].count().sort_values(ascending=False)[:30]

movie_id  title                                                
2858      American Beauty (1999)                                   3211
260       Star Wars: Episode IV - A New Hope (1977)                2910
1196      Star Wars: Episode V - The Empire Strikes Back (1980)    2885
1210      Star Wars: Episode VI - Return of the Jedi (1983)        2716
2028      Saving Private Ryan (1998)                               2561
589       Terminator 2: Judgment Day (1991)                        2509
593       Silence of the Lambs, The (1991)                         2498
1198      Raiders of the Lost Ark (1981)                           2473
1270      Back to the Future (1985)                                2460
2571      Matrix, The (1999)                                       2434
480       Jurassic Park (1993)                                     2413
2762      Sixth Sense, The (1999)                                  2385
608       Fargo (1996)                                             2371


In [15]:
# 가장 영화를 많이 본 유저 30명(시청 건수가 높은 순)
top_user_count = data.groupby('user_id')['movie_id'].count()
top_user_count.sort_values(ascending=False).head(30)

user_id
4169    1968
4277    1715
1680    1515
3618    1146
1015    1145
5831    1136
2909    1119
1941    1116
424     1106
1980    1054
3224    1034
1285    1034
3539    1029
3391    1012
3032    1003
1181     981
3841     976
4344     973
4448     970
1088     966
549      956
889      943
4725     936
678      933
1448     930
2063     906
3292     897
1698     883
4808     868
2116     855
Name: movie_id, dtype: int64

In [16]:
# 유저별 몇 개의 영화를 보는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [17]:
# 유저별 영화 시청 횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

### 내 데이터로 테스트해보기

In [18]:
# 내가 좋아하는 영화 제목 넣어보기, 
my_favorite = [ 'Fargo (1996)', 'Toy Story (1995)' ,'Braveheart (1995)' ,'Sixth Sense, The (1999)' ,'Pulp Fiction (1994)']

# 영화제목을 movie_id로 매핑하기
movie_title = dict(zip(data['title'], data['movie_id']))
movie_ids = [movie_title[i] for i in my_favorite]
movie_ids

# 6041이라는 user_id가 위 영화를 5회씩 시청했다고 입력하기
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': movie_ids , 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'yesica'이라는 데이터가 없다면
    data = data.append(my_playlist, ignore_index=True)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
else: 
    print('user_id exists!')
data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,6041,608,5,Fargo (1996)
836479,6041,1,5,Toy Story (1995)
836480,6041,110,5,Braveheart (1995)
836481,6041,2762,5,"Sixth Sense, The (1999)"
836482,6041,296,5,Pulp Fiction (1994)


In [19]:
data['movie_id'].nunique() # 영화갯수는 그래도인지 확인한다

3628

In [20]:
# data에 있는 유니크한 사용자 수가 늘었는지 확인한다
data['user_id'].nunique()  

6040

### 모델에 활용하기 위한 전처리 (실습)¶

In [21]:
# pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_title_unique = data['title'].unique()

# 유저, 영화를 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_title_unique)}

data

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
836478,6041,608,5,Fargo (1996)
836479,6041,1,5,Toy Story (1995)
836480,6041,110,5,Braveheart (1995)
836481,6041,2762,5,"Sixth Sense, The (1999)"


In [22]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 한다.
print(movie_to_idx['Fargo (1996)'])

6039
51


In [23]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

# 위의 인덱싱 작업을 통해서 내가 입력한 user_id가 6041에서 6039로 변경된 것을 볼 수 있다. 앞에 2개가 비어있었던 것임을 알 수 있다.
data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,608,5,51
836479,6039,1,5,40
836480,6039,110,5,87
836481,6039,2762,5,38


In [24]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 한다.
print(movie_to_idx['Fargo (1996)'])

6039
51


### 사용자의 명시적/암묵적 평가

In [25]:
# 우리의 데이터셋에서 1회만 영화를 시청한 데이터의 비율은 어떻게 될까요?
# 1회만 영화를 본 데이터의 비율을 보는 코드
only_one = data[data['count']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


## 4) CSR matrix를 직접 만들어 봅시다.

In [26]:
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape

# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()
num_movie_title = data['title'].nunique()

num_user, num_movie, num_movie_title

(6040, 3628, 3628)

In [27]:
csr_data = csr_matrix((data['count'].values, (data.user_id, data.title)), shape= (num_user, num_movie_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [28]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [30]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [31]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [32]:
# 모델 학습이 끝났습니다!
# 모델이 1)저의 벡터와 Fargo의 벡터를 어떻게 만들고 있는지와 
# 2)두 벡터를 곱하면 어떤 값이 나오는지 살펴보겠습니다.

In [33]:
movie_to_idx

{"One Flew Over the Cuckoo's Nest (1975)": 0,
 'James and the Giant Peach (1996)': 1,
 'My Fair Lady (1964)': 2,
 'Erin Brockovich (2000)': 3,
 "Bug's Life, A (1998)": 4,
 'Princess Bride, The (1987)': 5,
 'Ben-Hur (1959)': 6,
 'Christmas Story, A (1983)': 7,
 'Snow White and the Seven Dwarfs (1937)': 8,
 'Wizard of Oz, The (1939)': 9,
 'Beauty and the Beast (1991)': 10,
 'Gigi (1958)': 11,
 'Miracle on 34th Street (1947)': 12,
 "Ferris Bueller's Day Off (1986)": 13,
 'Sound of Music, The (1965)': 14,
 'Airplane! (1980)': 15,
 'Tarzan (1999)': 16,
 'Bambi (1942)': 17,
 'Awakenings (1990)': 18,
 'Big (1988)': 19,
 'Pleasantville (1998)': 20,
 'Wallace & Gromit: The Best of Aardman Animation (1996)': 21,
 'Back to the Future (1985)': 22,
 "Schindler's List (1993)": 23,
 'Meet Joe Black (1998)': 24,
 'Pocahontas (1995)': 25,
 'E.T. the Extra-Terrestrial (1982)': 26,
 'Titanic (1997)': 27,
 'Ponette (1996)': 28,
 'Close Shave, A (1995)': 29,
 'Antz (1998)': 30,
 'Girl, Interrupted (1999)':

In [34]:
yesica, Fargo = user_to_idx[6041], movie_to_idx['Fargo (1996)']
yesica_vector, Fargo_vector = als_model.user_factors[yesica], als_model.item_factors[Fargo]

In [35]:
yesica_vector

array([-0.13129069,  0.40852815, -0.35740373,  0.24134171,  0.3522464 ,
       -0.06791404, -1.027825  , -0.38137385, -0.02685145,  0.1393857 ,
        0.36734208,  0.90277785,  0.6522533 , -0.24324042, -1.0057639 ,
       -0.54283583, -0.24606165,  0.76006013,  0.16324146, -0.11331457,
        0.6666614 , -0.19725086, -0.36773086,  0.05417748, -0.39741397,
       -0.16600513,  0.145353  , -0.15283552, -0.2775566 ,  0.0186504 ,
       -0.3652658 ,  0.23278068,  0.39850488, -0.2334876 , -0.30312115,
        0.9594973 , -0.22233614, -0.6265373 ,  0.55917513, -0.21741217,
       -0.07472374, -0.5302751 ,  1.0768979 ,  0.01526338, -0.7457432 ,
       -0.30522653,  0.19997981,  0.74897796, -0.13731208, -0.06090926,
       -0.02414213,  0.05019025,  0.11485142, -1.0160239 , -0.14235932,
       -1.3707459 , -0.46299776,  0.7429666 ,  0.30421472, -0.08030193,
       -0.59594274, -0.4307238 ,  0.12921026, -0.05844112, -0.37551904,
        0.09511943,  0.31009054,  0.8365076 , -0.8022364 , -0.16

In [36]:
Fargo_vector

array([-3.86369159e-03, -8.39931984e-03, -9.96936392e-03,  8.53422284e-03,
        3.28216255e-02,  3.44967237e-03, -2.17137188e-02, -6.33101258e-03,
        1.46010816e-02, -2.63961498e-02,  1.90457571e-02,  2.46354546e-02,
        1.32977488e-02,  1.40352296e-02, -2.89024599e-02,  3.32487025e-03,
        6.39699306e-03,  2.14024168e-02,  2.15999093e-02, -2.05674898e-02,
        1.61626022e-02,  5.26934338e-04, -5.41465590e-03,  1.24572776e-02,
       -1.00834453e-02,  1.92619767e-02,  1.58580057e-02,  8.70232005e-03,
        1.74223687e-02, -5.75331447e-04, -1.87636968e-02, -2.45866645e-02,
        2.38329768e-02, -2.54171658e-02,  6.57530967e-03,  5.08317463e-02,
        1.32553130e-02,  2.17085965e-02,  2.80724261e-02,  1.04777869e-02,
       -5.85206272e-03, -1.63103100e-02,  3.02027632e-02,  1.26034674e-05,
        5.62306447e-03,  1.75030306e-02,  5.22580603e-03,  1.29597476e-02,
        1.89326275e-02,  3.93995130e-03,  3.02157383e-02,  1.23000164e-02,
        5.11050411e-03, -

In [37]:
# yesica와 Fargo를 내적하는 코드
np.dot(yesica_vector, Fargo_vector)

0.63758254

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [38]:
# 모델이 yesica의 Braveheart에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다
Braveheart = movie_to_idx['Braveheart (1995)']
Braveheart_vector = als_model.item_factors[Braveheart]
np.dot(yesica_vector, Braveheart_vector)

0.48353398

In [39]:
# 'Toy Story (1995)'와 비슷한 영화를 추천해준다.
favorite_movie = 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 0.19408692),
 (50, 0.15441471),
 (4, 0.11433132),
 (33, 0.11373418),
 (322, 0.106304124),
 (110, 0.10014886),
 (330, 0.0924502),
 (10, 0.08736635),
 (255, 0.08061872),
 (20, 0.07713828),
 (34, 0.07629687),
 (16, 0.0757042),
 (32, 0.07312702),
 (126, 0.07292574),
 (160, 0.06774406)]

In [40]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Babe (1995)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)",
 'Pleasantville (1998)',
 'Mulan (1998)',
 'Tarzan (1999)',
 'Hercules (1997)',
 'Shakespeare in Love (1998)',
 'Forrest Gump (1994)']

In [41]:
# 'Sixth Sense, The (1999)'와 비슷한 영화를 추천해준다.
favorite_movie = 'Sixth Sense, The (1999)'  # Sixth Sense, The (1999)
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(38, 0.21406253),
 (121, 0.10981776),
 (233, 0.103438094),
 (273, 0.095723726),
 (170, 0.090858966),
 (141, 0.089917436),
 (796, 0.08807225),
 (243, 0.08675505),
 (220, 0.085553214),
 (235, 0.08510645),
 (124, 0.08447961),
 (157, 0.08338865),
 (818, 0.08229368),
 (3591, 0.08115391),
 (1706, 0.080697864)]

In [42]:
#movie_to_idx 를 뒤집어, index로부터 영화 title을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Sixth Sense, The (1999)',
 'Silence of the Lambs, The (1991)',
 'Usual Suspects, The (1995)',
 'Fight Club (1999)',
 'Being John Malkovich (1999)',
 'Fugitive, The (1993)',
 'Bone Collector, The (1999)',
 'Ghostbusters (1984)',
 'Seven (Se7en) (1995)',
 'Sleepy Hollow (1999)',
 'Matrix, The (1999)',
 'Shawshank Redemption, The (1994)',
 'Stigmata (1999)',
 "Heaven's Burning (1997)",
 'Stuart Little (1999)']

In [43]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [44]:
# 'Matrix, The (1999)'와 비슷한 영화를 추천해준다.
get_similar_movie('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Face/Off (1997)',
 'Fifth Element, The (1997)',
 'Twelve Monkeys (1995)',
 'Jurassic Park (1993)',
 'Hunt for Red October, The (1990)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [45]:
# 내가 좋아할 만한 영화를 추천해준다
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(121, 0.66327906),
 (157, 0.597315),
 (23, 0.57990444),
 (48, 0.51934767),
 (50, 0.46173364),
 (99, 0.45476407),
 (233, 0.4459082),
 (269, 0.4279489),
 (248, 0.354133),
 (224, 0.3137439),
 (92, 0.3134439),
 (141, 0.29781377),
 (4, 0.2944352),
 (289, 0.28461748),
 (322, 0.27273256),
 (220, 0.2586512),
 (170, 0.25621277),
 (472, 0.24752188),
 (124, 0.24330378),
 (110, 0.2416741)]

In [46]:
# 내가 좋아할 만한 영화 타이틀들을 보여준다.
[idx_to_movie[i[0]] for i in movie_recommended]

['Silence of the Lambs, The (1991)',
 'Shawshank Redemption, The (1994)',
 "Schindler's List (1993)",
 'Saving Private Ryan (1998)',
 'Toy Story 2 (1999)',
 'American Beauty (1999)',
 'Usual Suspects, The (1995)',
 'GoodFellas (1990)',
 'Good Will Hunting (1997)',
 'L.A. Confidential (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Fugitive, The (1993)',
 "Bug's Life, A (1998)",
 'Reservoir Dogs (1992)',
 'Babe (1995)',
 'Seven (Se7en) (1995)',
 'Being John Malkovich (1999)',
 'Sling Blade (1996)',
 'Matrix, The (1999)',
 'Groundhog Day (1993)']

In [47]:
# Pulp Fiction (1994) 비슷한 영화를 추천해준다
Pulp_Fiction = movie_to_idx['Pulp Fiction (1994)'] # Pulp Fiction (1994)
explain = als_model.explain(user, csr_data, itemid=Pulp_Fiction)

In [48]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Pulp Fiction (1994)', 0.29985088780633784),
 ('Fargo (1996)', 0.2193270116197911),
 ('Sixth Sense, The (1999)', 0.05058031669480803),
 ('Toy Story (1995)', 0.00018020851485293942),
 ('Braveheart (1995)', -0.012199292523138207)]

In [49]:
# 입력한 영화와 유사한 영화를 추천해 준다